In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv

# pd.set_option('display.float_format', lambda x: '%.3f' % x)

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'


In [2]:
from epsampling.utils import load_csv

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
timestamp = '20240925-113849'

df_deaths,_ = load_csv('df_deaths_filt', data_dir=f'{data_dir}cached/', timestamp=timestamp)
df_acs,_ = load_csv('df_acs_filt', data_dir=f'{data_dir}cached/', timestamp=timestamp)
# df_acs.set_index('Fips',inplace=True,drop=True)

In [3]:
# display(df_deaths.head())

In [4]:
# df = df_deaths
# dates = sorted(df.Date.unique())

# dates

# lim = int(len(dates) * test_size)
# dates_test = dates[lim:]
# dates_train = dates[:lim]
# # print(len(dates_test), len(dates_train))
# df_train = df[df.Date.isin(dates_train)]
# df_test = df[df.Date.isin(dates_test)]

In [5]:
# from datetime import datetime
# past_date = datetime.strptime("2022-04-09", "%Y-%m-%d")
# now_date = datetime.strptime("2022-05-07", "%Y-%m-%d")
# print(now_date - past_date)

In [6]:
df = df_deaths.copy()
dates = sorted(df.Date.unique())

# now_to_pasts = {}
# for i,date in enumerate(dates):
#     if i < num_weeks_past:
#         continue
#     now = dates[i]
#     pasts = dates[i-10]
#     now_to_pasts[now] = pasts
    # print(now,past)
    # break
    # datetime.strptime("2022-05-07", "%Y-%m-%d")
# print(len(now_to_pasts))

# # # # # # # # # # # # # # # 
num_weeks_from = 3
num_weeks_to = 8
# # # # # # # # # # # # # # # 

for i in range(num_weeks_from, num_weeks_to):
    print(i)
    df[f'Weeks_ago_{i+1}'] = None
# df

3
4
5
6
7


In [7]:
from tqdm import tqdm

# # # # # # # # # # # # # # 
fipss = df.Fips.unique()
# fipss = fipss[:2]
# # # # # # # # # # # # # # 

dfs = []
for fips in tqdm(fipss,total=len(fipss)):
    df_county = df[df.Fips==fips] 
    df_county.sort_values('Date',inplace=True)
    df_county.reset_index(inplace=True,drop=True)

    for i in range(len(df_county)):

        if i < num_weeks_to:
            continue
        else:

            # # # # # # # # # # # # # # 
            from_date = i - num_weeks_from #num_weeks_from, 4
            to_date = i - num_weeks_to #num_weeks_to, 9
            # # # # # # # # # # # # # # 
            
            this = df_county[to_date:from_date]['True_county_deaths'].values[::-1]
            # print(this)

            # this = df_county[i-10:i]['True_county_deaths'].values
            wk_to_death = { f'Weeks_ago_{k+1}':v for (k,v) in zip(range(num_weeks_from,num_weeks_to),this) }

            # df_county.iloc[i]
            # this = df_county[i-10:i]['True_county_deaths'].values
            # wk_to_death = {f'Weeks_past_{k}':v for (k,v) in enumerate(this)}

            # print(wk_to_death)


            df_county.loc[i, wk_to_death.keys()] = wk_to_death.values()

        # display(df_county)
        # print(i)
        # break
    # break
        
    dfs.append(df_county)
    # print(len(df_county))
    # display(df_county)


100%|██████████| 3130/3130 [03:57<00:00, 13.18it/s]


In [8]:
len(dfs)

3130

In [11]:
df_tot = pd.concat(dfs)
# df_tot.reset_index(inplace=True,drop=True)
df_tot

,State_fips,State,County,Fips,Date,COVIDhubEns_state_deaths,Pop,Pop_ratio,True_county_deaths,Naive_county_deaths,Timestring,Weeks_ago_4,Weeks_ago_5,Weeks_ago_6,Weeks_ago_7,Weeks_ago_8
0,41,OR,Baker,41001,2020-05-09,125.655,16124,0.003823,0.0,0.480367,4334940,None,None,None,None,None
1,41,OR,Baker,41001,2020-05-16,143.950,16124,0.003823,0.0,0.550307,4335360,None,None,None,None,None
2,41,OR,Baker,41001,2020-05-23,150.378,16124,0.003823,0.0,0.574881,4335780,None,None,None,None,None
3,41,OR,Baker,41001,2020-05-30,159.093,16124,0.003823,0.0,0.608197,4336200,None,None,None,None,None
4,41,OR,Baker,41001,2020-06-06,164.199,16124,0.003823,0.0,0.627717,4338360,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,24,MD,Baltimore city,24510,2022-04-09,14367.000,593490,0.098168,1765.0,1410.374156,4504140,1752.0,1738.0,1726.0,1714.0,1693.0
103,24,MD,Baltimore city,24510,2022-04-16,14391.000,593490,0.098168,1771.0,1412.730179,4504560,1755.0,1752.0,1738.0,1726.0,1714.0
104,24,MD,Baltimore city,24510,2022-04-23,14423.000,593490,0.098168,1771.0,1415.871543,4504980,1762.0,1755.0,1752.0,1738.0,1726.0
105,24,MD,Baltimore city,24510,2022-04-30,14443.000,593490,0.098168,1773.0,1417.834895,4505400,1763.0,1762.0,1755.0,1752.0,1738.0


In [12]:
from datetime import datetime
d = datetime.today().strftime('%Y%m%d-%H%M%S')
df_tot.to_csv(f'{data_dir}cached/df_deaths_pastweeks_{d}.csv',index=False)
print(d)

20240925-132841
